In [2]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms

In [11]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform = transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

<class 'torchvision.datasets.mnist.FashionMNIST'>
